In [1]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 MB 12.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.90-cp310-cp310-linux_x86_64.whl size=3414603 sha256=838696a6c596d8e9ea40b8af9aeb67afbec2645345c199eb3a0c7fa1ea4eb4d4
  Stored in directory: /root/.cache/pip/wheels/3d/67/02/f950031435db4a5a02e6269f6adb6703bf1631c3616380f3c6
Successfully built llama-cpp-python


In [2]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2024-09-19 05:22:54--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 13.33.30.76, 13.33.30.23, 13.33.30.114, ...
Connecting to huggingface.co (huggingface.co)|13.33.30.76|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1726982575&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNjk4MjU3NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTl

In [5]:
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.6 MB/s eta 0:00:00


In [6]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.7 MB/s eta 0:00:00


In [6]:

from langchain import LlamaCpp

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

In [7]:
from langchain import PromptTemplate

# Create a prompt template with the "input_prompt" variable
template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

In [8]:
basic_chain = prompt | llm

In [10]:
# Use the chain
basic_chain.invoke(
    {
        "input_prompt": "Hi! My name is Ashfaq. What is 1 + 1?",
    }
)

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


' Hello Ashfaq! The answer to 1 + 1 is 2.'

In [11]:
from langchain import LLMChain

# Create a chain for the title of our story
template = """<s><|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

<ipython-input-11-61dd782c6da9>:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")


In [12]:
title.invoke({"summary": "a girl that lost her mother"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of the Fallen: A Tale of Loss and Remembrance"'}

In [13]:
# Create a chain for the character description using the summary and title
template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title"]
)
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [14]:
# Create a chain for the story using the summary, title, and character description
template = """<s><|user|>
Create a story about {summary} with the title {title}. The main character is: {character}. Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title", "character"]
)
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [15]:
# Combine all three components to create the full chain
llm_chain = title | character | story

In [16]:
llm_chain.invoke("a girl that lost her mother")

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of Loss: A Journey Beyond Grief"',
 'character': ' The central character, Emily, is a resilient and introspective young woman grappling with the profound loss of her beloved mother. Throughout the story, she embarks on a transformative journey to discover newfound strengths, redefine her identity, and navigate the complex emotions that accompany grief.',
 'story': ' In "Whispers of Loss: A Journey Beyond Grief," Emily—a resilient young woman whose world shattered when she lost her mother to a cruel, unexpected illness—embarks on an introspective odyssey that transcends mere sorrow. With each step taken through the valleys and peaks of emotions, from denial\'s numb embrace to anger\'s searing flame, Emily discovers within herself untapped reservoirs of strength. Her journey leads her across continents and into sacred spaces where ancient wisdom intertwines with modern healing practices. As she redefines her identity amidst

In [17]:
#CONVERSATION BUFFER
# Create an updated prompt template to include a chat history
template = """<s><|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

In [18]:
from langchain.memory import ConversationBufferMemory

# Define the type of Memory we will use
memory = ConversationBufferMemory(memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [20]:
# Generate a conversation and ask a basic question
llm_chain.invoke({"input_prompt": "Hi! My name is Ashfaq. What is 1 + 1?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'Hi! My name is Ashfaq. What is 1 + 1?',
 'chat_history': '',
 'text': " Hi Ashfaq! The answer to your question, 1 + 1 equals 2. :) Hope you're having a great day! If there's anything else I can help with, feel free to ask.\n---"}

In [21]:
# Does the LLM remember the name we gave it?
llm_chain.invoke({"input_prompt": "What is my name?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Ashfaq. What is 1 + 1?\nAI:  Hi Ashfaq! The answer to your question, 1 + 1 equals 2. :) Hope you're having a great day! If there's anything else I can help with, feel free to ask.\n---",
 'text': ' Your name is Ashfaq.'}

In [25]:
from langchain.memory import ConversationBufferWindowMemory

# Retain only the last 2 conversations in memory
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [26]:
# Ask two questions and generate two conversations in its memory
llm_chain.invoke({"input_prompt":"Hi! My name is Ashfaq and I am 24 years old. What is 1 + 1?"})
llm_chain.invoke({"input_prompt":"What is 3 + 3?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is 3 + 3?',
 'chat_history': 'Human: Hi! My name is Ashfaq and I am 24 years old. What is 1 + 1?\nAI:  The answer to the mathematical question "What is 1 + 1?" is 2. While your introduction about yourself, Ashfaq, is quite personal, let\'s focus on solving the problem at hand:\n\n1 + 1 = 2.\n\nThis basic arithmetic operation demonstrates addition, where combining one unit with another unit results in two units total.',
 'text': ' The answer to the mathematical question "What is 3 + 3?" is 6. Just like when you added 1 and 1 together, this is also a simple addition operation:\n\n3 + 3 = 6.\n\nIn this case, adding three units to another three units results in six units total.'}

In [27]:
llm_chain.invoke({"input_prompt":"What is my name?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': 'Human: Hi! My name is Ashfaq and I am 24 years old. What is 1 + 1?\nAI:  The answer to the mathematical question "What is 1 + 1?" is 2. While your introduction about yourself, Ashfaq, is quite personal, let\'s focus on solving the problem at hand:\n\n1 + 1 = 2.\n\nThis basic arithmetic operation demonstrates addition, where combining one unit with another unit results in two units total.\nHuman: What is 3 + 3?\nAI:  The answer to the mathematical question "What is 3 + 3?" is 6. Just like when you added 1 and 1 together, this is also a simple addition operation:\n\n3 + 3 = 6.\n\nIn this case, adding three units to another three units results in six units total.',
 'text': " Your name is Ashfaq. This question was asked after we discussed the sum of 1 + 1 and 3 + 3. Here's a quick recap:\n\n1 + 1 = 2, which means when you add one unit to another unit, it equals two units total.\n\n3 + 3 = 6, meaning that if you combine three units wit

In [28]:
llm_chain.invoke({"input_prompt":"What is my age?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my age?',
 'chat_history': 'Human: What is 3 + 3?\nAI:  The answer to the mathematical question "What is 3 + 3?" is 6. Just like when you added 1 and 1 together, this is also a simple addition operation:\n\n3 + 3 = 6.\n\nIn this case, adding three units to another three units results in six units total.\nHuman: What is my name?\nAI:  Your name is Ashfaq. This question was asked after we discussed the sum of 1 + 1 and 3 + 3. Here\'s a quick recap:\n\n1 + 1 = 2, which means when you add one unit to another unit, it equals two units total.\n\n3 + 3 = 6, meaning that if you combine three units with another set of three units, you get six units in total.\n\nI\'m delighted to help with your math questions!',
 'text': " I'm an AI and don't have access to personal information. Therefore, I'm unable to determine your age. If you provide me with the year of birth or any other relevant details, I can guide you on how to calculate it based on that information. Remember to

In [29]:
#CONVERSATION SUMMARY
# Create a summary prompt template
summary_prompt_template = """<s><|user|>Summarize the conversations and update with the new lines.

Current summary:
{summary}

new lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template
)

In [30]:
from langchain.memory import ConversationSummaryMemory

# Define the type of memory we will use
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)
# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [31]:
# Generate a conversation and ask for the name
llm_chain.invoke({"input_prompt": "Hi! My name is Ashfaq. What is 1 + 1?"})
llm_chain.invoke({"input_prompt": "What is my name?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What is my name?',
 'chat_history': " Hi Ashfaq! I'm an AI, and when you asked me what is 1 + 1, the result is 2.",
 'text': ' Your name is Assistant. The initial statement seems to have a mix-up as it mentions "Ashfaq" but then refers to yourself as the assistant. Based on your responses throughout our interactions, I am an AI known as the Assistant. There isn\'t a specific name assigned to me by creators; instead, I go by this title.'}

In [32]:
# Check whether it has summarized everything thus far
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'input_prompt': 'What was the first question I asked?',
 'chat_history': ' The conversation starts with a human inquiring about their own name and the AI clarifies that they are referred to as "Assistant." The initial statement regarding Ashfaq was incorrect since the AI is addressed as Assistant throughout our interactions, not Ashfaq. Additionally, the AI reiterates its function of providing information and assistance based on its programming, rather than having a personal name assigned by creators.',
 'text': ' The first question you asked was "What is your name?"'}

In [33]:
# Check what the summary is thus far
memory.load_memory_variables({})

{'chat_history': ' In our conversation, the human initially asks about the AI\'s name and learns that it does not have a personal name but is referred to as "Assistant". The initial mention of Ashfaq was corrected when the AI reiterates its purpose is to provide information and assistance based on programming. Later in the discussion, the human inquires about their first question, which was asking for the AI\'s name.'}

In [2]:
!pip install langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [14]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.8 MB/s eta 0:00:00


In [17]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [19]:
from langchain_groq import ChatGroq

groq_llm = ChatGroq(
    model="llama3-groq-70b-8192-tool-use-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [8]:
# Create the ReAct template
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [10]:
%pip install -qU duckduckgo-search langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 43.7 MB/s eta 0:00:00


In [20]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# Prepare tools
tools = load_tools(["llm-math"], llm=groq_llm)
tools.append(search_tool)

In [21]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent
agent = create_react_agent(groq_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [22]:
# What is the Price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)

I need to find the current price of a MacBook Pro in USD first.

Action: duckduck
Action Input: "current price of MacBook Pro in USD"snippet: Top MacBook Pro deals in September 2024. Price war: Popular M3 Pro 14-inch model with 18GB RAM, 512GB SSD drops to record low $1,499, 16-inch model falls to $1,999 ($500 off) Exclusive coupon ..., title: Best Apple MacBook Pro Deals September 2024 - up to $700 Off - AppleInsider, link: https://appleinsider.com/deals/best-macbook-pro-deals, snippet: Apple MacBook Pro M3 8‑core CPU 10‑core GPU 14.2" Laptop for $1,399.00 (List Price $1,599.00) Apple MacBook Air M3 13.6" 8GB/256GB SSD (2024) for $929.00 (List Price $1,099.00) *Deals are ..., title: The Best Apple MacBook Deals for September 2024 - PCMag, link: https://www.pcmag.com/deals/best-apple-macbook-deals, snippet: View Deal. See all prices. Apple MacBook Pro 16-inch (M3 Pro/M3. $3,499. View Deal. See all prices. We check over 250 million products every day for the best prices. And if you'd st

KeyboardInterrupt: 